In [ ]:
# Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Hyper-parameters
num_epochs = 16
batch_size = 4
learning_rate = 0.001

In [ ]:
# Data preparation
# Dataset contains PILImage images of range [0, 1].
# We transform them to Tensors of normalized range [-1, 1].
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]
)

In [ ]:
# Train test split
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# Define classes
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
# Implement ConvNet
class ConvNet(nn.Module):
  def __init__(self):
    super(ConvNet, self).__init__()
    # feature learning layers
    self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5)
    self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
    self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)

    # classification layers
    self.fc1 = nn.Linear(16*5*5, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))

    # flatten
    x = x.view(-1, 16*5*5)

    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)

    return x

model = ConvNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
# Training the model
n_total_steps = len(train_loader)

for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    # origin shape: [4, 3, 32, 32] = 4, 3, 1024
    # input layer = 3 input channels, 6 output channels, 5 kernel size
    images = images.to(device)
    labels = labels.to(device)

    # Forward pass
    outputs = model(images)
    loss = criterion(outputs, labels)

    # Bacward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Log
    if (i+1) % 2000 == 0:
      print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

print('Finished Training')

Epoch [1/16], Step [2000/12500], Loss: 2.2863
Epoch [1/16], Step [4000/12500], Loss: 2.2358
Epoch [1/16], Step [6000/12500], Loss: 2.2148
Epoch [1/16], Step [8000/12500], Loss: 2.1886
Epoch [1/16], Step [10000/12500], Loss: 2.2241
Epoch [1/16], Step [12000/12500], Loss: 1.9391
Epoch [2/16], Step [2000/12500], Loss: 2.2906
Epoch [2/16], Step [4000/12500], Loss: 1.6531
Epoch [2/16], Step [6000/12500], Loss: 2.0232
Epoch [2/16], Step [8000/12500], Loss: 2.1837
Epoch [2/16], Step [10000/12500], Loss: 2.3546
Epoch [2/16], Step [12000/12500], Loss: 1.5428
Epoch [3/16], Step [2000/12500], Loss: 1.6040
Epoch [3/16], Step [4000/12500], Loss: 1.1669
Epoch [3/16], Step [6000/12500], Loss: 1.6386
Epoch [3/16], Step [8000/12500], Loss: 1.0692
Epoch [3/16], Step [10000/12500], Loss: 1.4504
Epoch [3/16], Step [12000/12500], Loss: 1.9131
Epoch [4/16], Step [2000/12500], Loss: 1.2350
Epoch [4/16], Step [4000/12500], Loss: 0.9926
Epoch [4/16], Step [6000/12500], Loss: 1.4357
Epoch [4/16], Step [8000/125

In [ ]:
# Testing
with torch.no_grad():
  n_correct = 0
  n_samples = 0
  n_class_correct = [0 for i in range(10)]
  n_class_samples = [0 for i in range(10)]

  for images, labels in test_loader:
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)

    # max returns (value, index)
    _, predicted = torch.max(outputs, 1)
    n_samples += labels.size(0)
    n_correct += (predicted == labels).sum().item()

    for i in range(batch_size):
      label = labels[i]
      pred = predicted[i]
      if (label == pred):
        n_class_correct[label] += 1
      n_class_samples[label] += 1

  acc = 100.0 * n_correct / n_samples
  print(f'Accuracy of the network: {acc}%')

  for i in range(10):
    acc = 100.0 * n_class_correct[i] / n_class_samples[i]
    print(f'Accuracy of {classes[i]}: {acc}%')

Accuracy of the network: 63.11%
Accuracy of plane: 73.4%
Accuracy of car: 76.6%
Accuracy of bird: 49.7%
Accuracy of cat: 32.4%
Accuracy of deer: 54.8%
Accuracy of dog: 51.5%
Accuracy of frog: 82.8%
Accuracy of horse: 57.8%
Accuracy of ship: 84.2%
Accuracy of truck: 67.9%
